In [4]:
from mongoengine import *
import os
import pandas as pd
from pandas import Series,DataFrame
import datetime

## 1 读取文件

### 读取方式1：使用pandas

In [5]:
targetfilename=r"5622.txt"
# 家中mac
targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
# 单位mac
# targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [6]:
fullname

'/Users/casablanca/03project/typhoonSearchSys/demo_data/5622.txt'

不使用pandas的read_table

In [7]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\n',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [8]:
data.head(8)

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...
5,4 121 109 90 71 58 53 59 73 87 96 ...
6,30 38 47 45 36 32 40 51 50 58 45 ...
7,9 156 159 142 126 106 100 98 -- -- -- ...


## 2 按行进行切分

In [6]:
len(data)

329

In [9]:
mark_list=[]
mark_list=[3,
 21,
 37,
 54,
 72,
 89,
 107,
 124,
 142,
 159,
 176,
 193,
 210,
 227,
 244,
 262,
 279,
 297,
 314]

In [10]:
# mark_list

In [11]:
step_interval=

SyntaxError: invalid syntax (<ipython-input-11-81ea31d0963b>, line 1)

In [13]:
data.iloc[21-3][0]

'1'

In [17]:
type(data[:37][0])

pandas.core.series.Series

In [18]:
type(data[:37])

pandas.core.frame.DataFrame

- 注意data[:37]本身是一个dataframe，是一个columns只有一列的df

In [20]:
data[:37].head()

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...


In [22]:
data[:37][0]

0                                                     1
1             S.N.   122      5622      9   4     QI...
2             0   1   2   3   4   5   6   7   8   9 ...
3     +                                             ...
4       9   159 156 135 107  90  93 110 123 145 141 ...
5       4   121 109  90  71  58  53  59  73  87  96 ...
6      30    38  47  45  36  32  40  51  50  58  45 ...
7       9   156 159 142 126 106 100  98  --  --  -- ...
8       5   130 129 115  95  77  67  65  74  89 101 ...
9      HM    26  30  27  31  29  33  33  --  --  -- ...
10                                                  ...
11      9   188 193 187 165 139 122 111 110 115 111 ...
12      6   131 137 135 119  99  84  78  81  93 108 ...
13     02    57  56  52  46  40  38  33  29  22   3 ...
14                                                  ...
15      9    --  --  --  --  --  --  --  --  --  -- ...
16      7   127 132 140 135 118 100  92  91  98 112 ...
17     03    --  --  --  --  --  --  --  --  -- 

找到截取的数据

In [23]:
data[21-3:37][0]

18                                                    1
19            S.N.     0      5622      9   2     LI...
20            0   1   2   3   4   5   6   7   8   9 ...
21    +                                             ...
22      9   379 346 366 254 201 151 109  97 117 173 ...
23      2   395 370 329 281 231 183 146 133 150 199 ...
24     28   -16 -24  37 -27 -30 -32 -37 -36 -33 -26 ...
25      9   403 405 384 342 289 226 172 121 103 117 ...
26      3   398 406 387 346 294 238 181 134 110 122 ...
27     29     5  -1  -3  -4  -5 -12  -9 -13  -7  -5 ...
28      9   373 419 425 429 393 334 264 196 135 107 ...
29      4   360 405 421 401 355 298 234 171 118  92 ...
30     30    13  14   4  28  38  36  30  25  17  15 ...
31      9   352 433 465 460 429 381 322 253 196 145 ...
32      5   295 370 422 437 411 359 296 227 162 110 ...
33     HM    57  63  43  23  18  22  26  26  34  35 ...
34              MAX                                 ...
35             S.N. 37950      5622      9   4  